<a href="https://colab.research.google.com/github/sudhir2016/Google-Colab-13/blob/master/Social_dist_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load cv2 numpy and matplotlib.

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Read and show image.

In [0]:
img=cv2.imread('India_image3.jpg')

In [0]:
plt.imshow(img)

resize image and create a blob.

In [0]:
img = cv2.resize(img,None,fx=0.4, fy=0.4)

In [0]:
height,width,channels=img.shape

In [0]:
blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416))

You can use this command to get yolo3 weights. You also need coco names and yolo3 config files.

In [0]:
#!wget https://pjreddie.com/media/files/yolov3.weights

Get 80 object labels from coco.names

In [0]:
label=[]

In [0]:
file=open('coco.names')
for line in file:
  line1=line.strip()
  label.append(line1)

In [0]:
print(label)

In [0]:
len(label)

Prepare yolo3 model.

In [0]:
net = cv2.dnn.readNet("/content/drive/My Drive/yolov3.weights", "yolov3.cfg")

Feed blob to model.

In [0]:
net.setInput(blob)

Identify output layers of model, run blob on model and retrieve output.

In [0]:
output_layers=net.getUnconnectedOutLayersNames()

In [0]:
print(output_layers)

In [0]:
output = net.forward(output_layers)

In [0]:
len(output)

Define empty lists for bounding boxes and their X cordinates.

In [0]:
X=[]
boxes=[]

Object identified as 'person' has a class ID of 0. Retrieve all bounding box parameters for each person and append in list 'boxes'. List 'X' will contain the X cordinates for each such box.

In [0]:
for i in output:
    for j in i:
        score = j[5:]
        class_id = np.argmax(score)
        confidence = score[class_id]
        if (confidence > 0.5 and class_id==0):           
            center_x = int(j[0] * width)
            center_y = int(j[1] * height)
            w = int(j[2] * width)
            h = int(j[3] * height)           
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])            
            X.append(x)

Ignoring non max supression as this is a simple image.

In [0]:
#indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [0]:
len(boxes)
len(X)

Define empty list 'nsd' for identifying persons who are not in social distance.

In [0]:
nsd=[]

Use X cordinates of all boxes to calculate distance between persons and identify those not in social distance.

In [0]:
for i in range (3):
  for j in range (3):   
      X1=X[i]
      X2=X[j]
      dist=abs(X1-X2)     
      if dist<200 and dist !=0:        
          nsd.append(i)

In [0]:
print(nsd)

In [0]:
nsd=list(dict.fromkeys(nsd))

In [0]:
font = cv2.FONT_HERSHEY_SIMPLEX

Draw all bounding boxes on image in green.

In [0]:
for i in range(len(boxes)):
    x, y, w, h = boxes[i]   
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0))    

Convert bounding boxes for those who are not in social distance, to red.

In [0]:
for i in range(len(boxes)):
    x, y, w, h = boxes[i]  
    for j in range (len(nsd)):
      k=nsd[j]
      if k==i:
         cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,255))     
      

In [0]:
plt.imshow(img)

In [0]:
cv2.imwrite('out.jpg',img)